In [1]:
import json
import Levenshtein as lvs

In [2]:
doc = json.load(fp=open("../etc/b2find_disciplines.json"))
len(doc["disciplines"])

336

In [3]:
discs = {}
for value in doc["disciplines"]:
    parts = value.split("#")
    if len(parts) < 3:
        continue
    key = parts[2].strip()
    discs[key] = value
len(discs.keys())

335

In [4]:
# discs

In [5]:
def classify(text):
    for disc in discs:
        similarity = lvs.ratio(text.lower(), disc.lower())
        if similarity >= 0.9:
            print(disc, similarity)

In [6]:
classify("astrophysics and astronomy")

Astrophysics and Astronomy 1.0


In [7]:
classify("physics")

Physics 1.0


In [8]:
classify("information science")

Information Science 1.0


In [9]:
classify("Engineering")

Engineering 1.0
